In [3]:
import whisperx
import gc 

device = "cuda" 
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("ct2_output", device, compute_type=compute_type)


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.2.2+cu121. Bad things might happen unless you revert torch to 1.x.


In [9]:
audio = whisperx.load_audio("white_noise.mp3")
result = model.transcribe(audio, batch_size=batch_size, language="de")
print(result["segments"]) # before alignment

No active speech found in audio
[]


In [7]:
import pysubs2

subs = pysubs2.load_from_whisper(result)
subs.save("Schönleber_whisperx.srt")